As we kept the clusters in the data, we can dp the supervisied learning

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings("ignore")

cluster_data_set=pd.read_csv("clustered_datas.csv")
cluster_data_set.head()



,volume,marketCap,timestamp,crypto_name,high_low_spread,close_open_diff,crypto_encoded,day_of_week,day_sin,day_cos,cluster
0,0.0,1.288693e+09,2013-05-05 00:00:00+00:00,Bitcoin,5.242289,0.145647,8,6,-0.781831,0.62349,2
1,0.0,6.229819e+07,2013-05-05 00:00:00+00:00,Litecoin,0.346400,0.097760,30,6,-0.781831,0.62349,0
2,0.0,1.249023e+09,2013-05-06 00:00:00+00:00,Bitcoin,5.242289,-0.139802,8,0,0.000000,1.00000,2
3,0.0,5.859436e+07,2013-05-06 00:00:00+00:00,Litecoin,0.665000,-0.139802,30,0,0.000000,1.00000,0
4,0.0,1.240594e+09,2013-05-07 00:00:00+00:00,Bitcoin,5.242289,-0.139802,8,1,0.781831,0.62349,2


In [2]:
cluster_data_set.shape

(72946, 11)

In [3]:
cluster_means_sorted=cluster_data_set.groupby(cluster_data_set["cluster"])['high_low_spread'].mean().sort_values()

In [4]:
cluster_means_sorted

,high_low_spread
cluster,
4,0.077553
0,0.291461
5,1.971474
2,3.206033
6,5.090289
1,5.242289
3,5.242289


In [5]:
# mapping from cluster ID to volatility category
mapping = {
    4: 'low',
    0: 'low',
    5: 'medium',
    2: 'medium',
    6: 'high',
    1: 'high',
    3: 'high'
}

cluster_data_set['volatility_label'] = cluster_data_set['cluster'].map(mapping)
cluster_data_set.head()

,volume,marketCap,timestamp,crypto_name,high_low_spread,close_open_diff,crypto_encoded,day_of_week,day_sin,day_cos,cluster,volatility_label
0,0.0,1.288693e+09,2013-05-05 00:00:00+00:00,Bitcoin,5.242289,0.145647,8,6,-0.781831,0.62349,2,medium
1,0.0,6.229819e+07,2013-05-05 00:00:00+00:00,Litecoin,0.346400,0.097760,30,6,-0.781831,0.62349,0,low
2,0.0,1.249023e+09,2013-05-06 00:00:00+00:00,Bitcoin,5.242289,-0.139802,8,0,0.000000,1.00000,2,medium
3,0.0,5.859436e+07,2013-05-06 00:00:00+00:00,Litecoin,0.665000,-0.139802,30,0,0.000000,1.00000,0,low
4,0.0,1.240594e+09,2013-05-07 00:00:00+00:00,Bitcoin,5.242289,-0.139802,8,1,0.781831,0.62349,2,medium


now we take the clusterd data copy and delete the "cluster" column as we have the volatality_label

In [6]:
cluster_data_set_copy=cluster_data_set.copy()
cluster_data_set.drop("cluster",axis=1,inplace=True)

In [14]:
cluster_data_set.drop("timestamp",axis=1,inplace=True)

In [22]:
cluster_data_set.drop("crypto_name",axis=1,inplace=True)

In [23]:
cluster_data_set.head()

,volume,marketCap,high_low_spread,close_open_diff,crypto_encoded,day_of_week,day_sin,day_cos,volatility_label
0,0.0,1.288693e+09,5.242289,0.145647,8,6,-0.781831,0.62349,medium
1,0.0,6.229819e+07,0.346400,0.097760,30,6,-0.781831,0.62349,low
2,0.0,1.249023e+09,5.242289,-0.139802,8,0,0.000000,1.00000,medium
3,0.0,5.859436e+07,0.665000,-0.139802,30,0,0.000000,1.00000,low
4,0.0,1.240594e+09,5.242289,-0.139802,8,1,0.781831,0.62349,medium


In [24]:
cluster_data_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72946 entries, 0 to 72945
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   volume            72946 non-null  float64
 1   marketCap         72946 non-null  float64
 2   high_low_spread   72946 non-null  float64
 3   close_open_diff   72946 non-null  float64
 4   crypto_encoded    72946 non-null  int64  
 5   day_of_week       72946 non-null  int64  
 6   day_sin           72946 non-null  float64
 7   day_cos           72946 non-null  float64
 8   volatility_label  72946 non-null  object 
dtypes: float64(6), int64(2), object(1)
memory usage: 5.0+ MB


In [25]:
X = cluster_data_set.drop(columns=['volatility_label'])  # predictors
y = cluster_data_set['volatility_label']                           # target

print(X)
print(y)

             volume     marketCap  high_low_spread  close_open_diff  \
0      0.000000e+00  1.288693e+09         5.242289         0.145647   
1      0.000000e+00  6.229819e+07         0.346400         0.097760   
2      0.000000e+00  1.249023e+09         5.242289        -0.139802   
3      0.000000e+00  5.859436e+07         0.665000        -0.139802   
4      0.000000e+00  1.240594e+09         5.242289        -0.139802   
...             ...           ...              ...              ...   
72941  4.040134e+07  1.652957e+09         0.000791         0.000192   
72942  2.844351e+07  1.572825e+09         0.095049         0.049634   
72943  1.069497e+08  1.559551e+09         0.203285         0.145647   
72944  2.143268e+08  1.576291e+09         0.000017         0.000007   
72945  9.509743e+08  2.339868e+10         0.017568         0.003543   

       crypto_encoded  day_of_week   day_sin  day_cos  
0                   8            6 -0.781831  0.62349  
1                  30            6 

In [26]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier,AdaBoostClassifier
from sklearn.metrics import classification_report,accuracy_score
from xgboost import XGBClassifier

models = {
    'RandomForest': RandomForestClassifier(),
    'LogisticRegression': LogisticRegression(),
    'SVM': SVC(),
    'KNN': KNeighborsClassifier(),
    'Decision Tree' : DecisionTreeClassifier(),
    'KNN Algorithm' : KNeighborsClassifier(),
    'Gradient Boosting' : GradientBoostingClassifier(),
    'AdaBoostClassifier' : AdaBoostClassifier(),

}


In [27]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=42, stratify=y)
print(X_train)
print(y_train)

             volume     marketCap  high_low_spread  close_open_diff  \
34549  1.308295e+08  9.293685e+08         0.071653         0.044475   
55748  1.837988e+08  2.708772e+09         2.390704         0.145647   
36376  4.719266e+05  8.974699e+06         0.008291        -0.004070   
7895   3.505990e+06  8.054742e+08         0.190280        -0.088321   
63295  4.474944e+08  8.694313e+09         0.013717        -0.000367   
...             ...           ...              ...              ...   
50283  3.093483e+08  8.096961e+08         0.072998        -0.012978   
66054  1.149941e+08  5.349923e+09         0.024463         0.020615   
17883  1.673630e+08  1.297313e+10         0.007062         0.000580   
72710  2.757773e+08  2.695322e+09         0.020276         0.004796   
22213  3.633361e+06  2.923618e+07         0.000649         0.000072   

       crypto_encoded  day_of_week   day_sin   day_cos  
34549              13            1  0.781831  0.623490  
55748               4            

In [28]:

for i in range(len(models)):
  model=list(models.values())[i]
  print(model)

RandomForestClassifier()
LogisticRegression()
SVC()
KNeighborsClassifier()
DecisionTreeClassifier()
KNeighborsClassifier()
GradientBoostingClassifier()
AdaBoostClassifier()


In [29]:
model_score_list={}
for i in range(len(models)):
  model=list(models.values())[i]
  model.fit(X_train,y_train)
  y_pred=model.predict(X_test)
  score=accuracy_score(y_test,y_pred)
  model_score_list[list(models.keys())[i]]=score

  print(f"model name-{list(models.keys())[i]}")
  print(f"score - {score}")




model name-RandomForest
score - 0.9890335846470185
model name-LogisticRegression
score - 0.2796435915010281
model name-SVM
score - 0.7420150788211104
model name-KNN
score - 0.7543522960932145
model name-Decision Tree
score - 0.9837559972583961
model name-KNN Algorithm
score - 0.7543522960932145
model name-Gradient Boosting
score - 0.9859492803289924
model name-AdaBoostClassifier
score - 0.9271418779986292


RandomForest is giving the best score

In [31]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier

param_dist = {
    'n_estimators': [50, 60, 70],
    'max_depth': [None, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']
}

clf = RandomForestClassifier(random_state=42)

random_search = RandomizedSearchCV(
    clf,
    param_distributions=param_dist,
    n_iter=20,
    cv=3,
    scoring='accuracy',
    n_jobs=-1,
    random_state=42
)

random_search.fit(X_train, y_train)
print("Best params:", random_search.best_params_)
print("Best CV score:", random_search.best_score_)


Best params: {'n_estimators': 70, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'log2', 'max_depth': None}
Best CV score: 0.9897525532935774


In [32]:
yp=random_search.predict(X_test)
print(f"Accuracy Score{accuracy_score(y_test,yp)}")
print(f"Classification report{classification_report(y_test,yp)}")


Accuracy Score0.9898560657984922
Classification report              precision    recall  f1-score   support

        high       0.99      0.98      0.99      1385
         low       0.99      0.99      0.99      8707
      medium       0.98      0.98      0.98      4498

    accuracy                           0.99     14590
   macro avg       0.99      0.99      0.99     14590
weighted avg       0.99      0.99      0.99     14590

